## Map-Reduce

* Polynomial calculation
* Matrix multiplication

### Describe how a scheduler works

Scheduler has
* a queue of tasks

takes into account:
* has to fit into shared memory
* has to fit into registers - if not it has to spill registers to higher level memory
* tries to utilize available cores/SIMDs
* compiled kernel size

Reduction
* suitable for assosiative operators
* floating point operation order matters - not assosiative
* uses more memory - O(n) - to store intermediate results, serial version uses registers - O(1)
* O(log(n)) iterations
* can be composed of parallel block - each of them with serial loop inside
- in GPU each threads can access part of a cache line

## Levels of parallelism

* machine level
* device level
* processor level
* thread level
* vectorisation
* instruction level

### Shared Virtual Memory

If OpenCL 2.0 is available it is possible to use one pointer for both host and gpu memory - similarly to CUDA managed memory.

## OpenCL to CUDA dictionary


"OpenCL CUDA
Grid Grid
Work Group Block
Work Item Thread
kernel global
global device
local shared
private local
imagend t texture<type, n, ...>
barrier(LMF) syncthreads()
get local id(012) threadIdx.xyz
get group id(012) blockIdx.xyz
get global id(012) – (reimplement)"

## Exercise - Matrix Multiplication

So far we have used only one dimension of global and local work group size. For some problems it may be more convenient to think about it in two or three dimensions. Global and local work group sizes can have up to three dimensions. This limitation originates in computer graphics from which compute kernels evolved. The other name for compute kernels is shader - because the first programmable graphics pipeline stage was used for 'shading' - or illuminating - pixels.

Your task in this exercise is to move the implementation of square matrix multiplication from python to GPU. Square matrix has the same number of rows and columns.
* add a kernel that will correctly multiply two matrices
* use two dimensions when addressing threads in GPU - for local and global size
* compare execution with with the CPU version - think about what you are comparing

Write you code in [this](./matrix_multiplication.py) file. To check your code execute the cell below.

In [ ]:
!python matrix_multiplication.py --n 1024 --threads 8

Refer to the [solution](./matrix_multiplication_solution.py) if you get stuck.

Intel OneAPI

In [2]:
#include <vector>
#include <CL/sycl.hpp>

#define SIZE 1024

namespace sycl = cl::sycl;

int main() {
  std::array<int, SIZE> a, b, c;

  for (int i = 0; i<SIZE; ++i) {
    a[i] = i;
    b[i] = -i;
    c[i] = i;
  }

  {
  sycl::range<1> a_size{SIZE};

  auto platforms = sycl::platform::get_platforms();
 
 for (auto &platform : platforms) {

    std::cout << "Platform: "
      << platform.get_info<sycl::info::platform::name>()
      << std::endl;


    auto devices = platform.get_devices();
    for (auto &device : devices ) {
      std::cout << "  Device: "
        << device.get_info<sycl::info::device::name>()
        << std::endl;
    }

  }

  sycl::default_selector device_selector;
  sycl::queue d_queue(device_selector);

  sycl::buffer<int, 1>  a_device(a.data(), a_size);
  sycl::buffer<int, 1>  b_device(b.data(), a_size);
  sycl::buffer<int, 1>  c_device(c.data(), a_size);

  d_queue.submit([&](sycl::handler &cgh) {
    auto c_res = c_device.get_access<sycl::access::mode::write>(cgh);
    auto a_in = a_device.get_access<sycl::access::mode::read>(cgh);
    auto b_in = b_device.get_access<sycl::access::mode::read>(cgh);

    cgh.parallel_for<class ex1>(a_size,[=](sycl::id<1> idx) {
      c_res[idx] = a_in[idx] + b_in[idx];
    });

  });

  }

}

SyntaxError: invalid syntax (<ipython-input-2-5bf5cd1824cb>, line 6)